# Buy-and-Hold Strategy with Advance/Decline Line

In [1]:
# Import basic libraries for manipulating data.

# Please refer to xarray.pydata.org for xarray documentation.

# xarray works optimally with N-dimensional datasets in Python
# and is well suited for financial datasets with labels "time",
# "field" and "asset". xarray data structures can also be easily
# converted to pandas dataframes.

import xarray as xr
import numpy as np
import pandas as pd

# Import quantiacs libraries.

import qnt.data as qndata    # data loading and manipulation
import qnt.stats as qnstats  # key statistics
import qnt.graph as qngraph  # graphical tools
import qnt.ta as qnta        # indicator library

# display function for fancy displaying:
from IPython.display import display

# plotly lib for charts:
import plotly.graph_objs as go

# time lib:
import time
import datetime as dt

## Loading Data

Strategies must have a minimal **3-year in-sample track record** in order to be accepted for live evaluation after submission and they must define allocation weights for all assets every day during the live evaluation itself.

Allocation weights are the fractions of capital the strategy is investing in the loaded assets. The Quantiacs library follows the convention of positive allocation weights for long positions, negative allocation weights for short positions and vanishing allocation weights for those assets the strategy is not exposed to.

The sum of the absolute values of the allocation weights must be smaller or equal to 1, otherwise the Quantiacs library will automatically scale down the allocation weights so that the sum of their absolute values is equal to 1.

The load function provided by the Quantiacs library allows the user to define a time interval for loading the data:

* min_date defines the initial point of the simulation and it must be at least 3 years before the current date;

* max_date defines the final point of the simulation. It can be used for developing the system on a limited in-sample period. The system can later be tested on out-of-sample data between max_date and the current date. 

Once development has been completed it is mandatory to **remove any reference to max_date** (for example, commenting the correspondent line) so that the submission will run on live data.

In [2]:
data = qndata.load_data(tail=dt.timedelta(days=8*365),
                        dims=("time", "field", "asset"), # DataArray coordinates
                        forward_order=True               # Load data in ascending order
                       )

fetched chunk 1/11 1s
fetched chunk 2/11 2s
fetched chunk 3/11 2s
fetched chunk 4/11 3s
fetched chunk 5/11 4s
fetched chunk 6/11 5s
fetched chunk 7/11 5s
fetched chunk 8/11 6s
fetched chunk 9/11 7s
fetched chunk 10/11 8s
fetched chunk 11/11 8s
Data loaded 8s


## Buy-and-Hold Strategy

Here a buy-and-hold strategy on liquid assets is defined. The strategy defines equal positive allocation weights on all liquid assets and is fully invested.

It is important to **trade only liquid assets**, as defined by the corresponding Quantiacs filter function, otherwise the submission will not be accepted for live evaluation.

After strategy definition local (i.e. their value corresponds to the given date) key statistical indicators are computed:

 * equity: the value of the equity curve, i.e. the cumulative sum of profits and losses;
 * relative_return: the relative return of the strategy;
 * underwater: the value of the underwater chart, i.e. the chart tracking peak-to-through losses before a new peak is achieved; 
 * bias: the ratio of the difference between unsigned long and short allocation weights and their sum.
 
Global statistics are also shown:

* max_drawdown: the largest value in absolute sense of the underwater chart during all simulation interval;
* instruments: the number of instruments which got allocation for at least 1 day during all simulation interval.

The most interesting statistics are those evaluated on a rolling window of 3 years:

* volatility: the standard deviation of the relative returns;
* mean_return: the mean value of the relative returns;
* sharpe_ratio: the ratio of mean_return and volatility;
* avg_turnover: the average daily turnover. 

A submission will be accepted only if its **Sharpe ratio is larger than 1 over the last 3 years** at submission time.

In [3]:
output = data.sel(field="is_liquid") # 1 <-> asset is liquid; 0 otherwise

# normalize allocation weights so that the sum of absolute values = 1:
output = output / output.sum("asset")

stats = qnstats.calc_stat(data, output)

display(stats.to_pandas().tail())

equity_curve = stats.loc[:,"equity"]

# draw performance chart:
fig = go.Figure(data = [
    go.Scatter(
        x=equity_curve.time.to_pandas(),
        y=equity_curve,
        hovertext="Equity curve",
    )
])
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-08-13,2.421505,0.002087,0.238334,-0.000245,-0.381815,0.474343,0.113052,1.0,1069.0,0.026518,138.316682
2020-08-14,2.419542,-0.000811,0.238286,-0.001056,-0.381815,0.459263,0.109436,1.0,1069.0,0.026522,138.316682
2020-08-17,2.432978,0.005553,0.238296,0.000000,-0.381815,0.472033,0.112484,1.0,1069.0,0.026522,138.316682
2020-08-18,2.427545,-0.002233,0.238299,-0.002233,-0.381815,0.464791,0.110759,1.0,1069.0,0.026533,138.316682
2020-08-19,2.416222,-0.004664,0.238113,-0.006887,-0.381815,0.483905,0.115224,1.0,1069.0,0.026530,305.070310


## Buy-and-Hold Strategy with Advance/Decline Line

The buy-and-hold-strategy has problems when the overall stock market is in a bearish phase and is going down. The strategy can be improved using an indicator known as Advance/Decline line, which measures the number of instruments participating in a rise/fall of the stock market:

adl(today) = adl(yesterday) + nr. advancing instruments - nr. declining instruments

In [4]:
# define Advance/Decline Line using "close" prices:
adl = qnta.ad_line(data.sel(field="close")) * 1.0

# smooth line using an exponential moving average:
adl_ma = qnta.ema(adl, 110)

# consider the variation of the previous indicator:
adl_ma_ch = adl_ma - adl_ma.shift(time=13)

# define a positive trend for positive values:
positive_trend = adl_ma_ch > 0

# draw adl chart and visualize if strategy catches positive trends:
fig = go.Figure(data = [
    go.Scatter(
        x=adl_ma.time.to_pandas(),
        y=adl_ma,
        name="adl_ma",
        line= dict(width=1,color="red")
    ),
    go.Scatter(
        x=adl_ma.time.to_pandas(),
        y=adl_ma.where(positive_trend),
        name="positive trend",
        line = dict(width=2,color="green")
    )
])
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

# calculate output using the same liquidity filter of the buy-and-hold strategy
# supplemented by the positive trend condition:
output = data.sel(field="is_liquid")
output = output.where(positive_trend)
output = output / output.sum("asset")

stats = qnstats.calc_stat(data, output)

display(stats.to_pandas().tail())

equity_curve = stats.loc[:,"equity"]

# draw performance chart:
fig = go.Figure(data = [
    go.Scatter(
        x=equity_curve.time.to_pandas(),
        y=equity_curve,
        hovertext="Equity curve",
    )
])
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-08-13,1.857392,0.002110,0.161050,-0.054057,-0.238952,0.331273,0.053351,1.0,1059.0,0.032514,119.078610
2020-08-14,1.855870,-0.000819,0.160972,-0.054832,-0.238952,0.310153,0.049926,1.0,1059.0,0.032519,119.078610
2020-08-17,1.866288,0.005613,0.160992,-0.049526,-0.238952,0.328159,0.052831,1.0,1059.0,0.032519,119.078610
2020-08-18,1.862075,-0.002257,0.160994,-0.051672,-0.238952,0.317966,0.051191,1.0,1059.0,0.032530,119.078610
2020-08-19,1.853296,-0.004715,0.160726,-0.056143,-0.238952,0.344674,0.055398,1.0,1059.0,0.032527,112.300531


## Final calculations

In [5]:
data = qndata.load_data(tail=dt.timedelta(days=4*365),
                        dims=("time", "field", "asset"), # DataArray coordinates
                        forward_order=True               # Load data in ascending order
                       )
adl = qnta.ad_line(data.sel(field="close")) * 1.0
adl_ma = qnta.ema(adl, 110)

adl_ma_ch = adl_ma - adl_ma.shift(time=13)
positive_trend = adl_ma_ch > 0

output = data.sel(field="is_liquid")
output = output.where(positive_trend)
output = output / output.sum("asset")

stats = qnstats.calc_stat(data, output)

fetched chunk 1/5 1s
fetched chunk 2/5 3s
fetched chunk 3/5 4s
fetched chunk 4/5 6s
fetched chunk 5/5 7s
Data loaded 7s


## Plots

In [6]:
# show plot with profit and losses:
performance = stats.to_pandas()["equity"].iloc[(252*3):]
qngraph.make_plot_filled(performance.index, performance, name="Equity Curve", type="log")

In [7]:
# show underwater chart:
UWchart = stats.to_pandas()["underwater"].iloc[(252*3):]
qngraph.make_plot_filled(UWchart.index, UWchart, color="darkred", name="Underwater Chart", range_max=0)

In [8]:
# show rolling Sharpe ratio on a 3-year basis:
SRchart = stats.to_pandas()["sharpe_ratio"].iloc[(252*3):]
qngraph.make_plot_filled(SRchart.index, SRchart, color="#F442C5", name="Rolling Share Ratio")

In [9]:
# show bias chart:
biaschart = stats.to_pandas()["bias"].iloc[(252*3):]
qngraph.make_plot_filled(biaschart.index, biaschart, color="#5A6351", name="Bias Chart")

## Check Correlations

The following check is important before submission, as the correlation of the strategy over the last 3 years with all strategies running on the Quantiacs platform with a higher Sharpe ratio over the last 3 years **must be smaller than 0.8** at submission time.

In [10]:
qnstats.print_correlation(output, data)


Ok. This strategy does not correlate with other strategies.


## Write Output

The write output step is mandatory for submission.

In [11]:
qndata.write_output(output)

write output: /root/fractions.nc.gz
